In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [2]:
# Dictionaries for factories (supply in tons), depots(throughput in tons), customers(demand in tons)
f = dict({
    'Liverpool': 150000,
    'Brighton': 200000
})

d = ({
    'Newcastle': 70000,
    'Birmingham': 50000,
    'London': 100000,
    'Exeter': 40000
})

c = ({
    'C1': 50000,
    'C2': 10000,
    'C3': 40000,
    'C4': 35000,
    'C5': 60000,
    'C6': 20000
})

### Check the weight 0.4 for objective function 2

In [3]:
w=0.4
# Dictionary for distribution costs per factory (in €/ton) for the sum of 2 objective functions 
arcs, distr_cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0+w,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'C1'): 2.0+w,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5+w,
    ('Newcastle', 'C6'): 1.0+w,
    ('Birmingham', 'C1'): 1.0+w,
    ('Birmingham', 'C2'): 0.5+w,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5+w,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5+w,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5+w,
    ('Exeter', 'C6'): 1.5    
})

In [4]:
# Define model
model = Model('Transshipment_Assignment')

# Create decision variables
plan = model.addVars(arcs, name="plan")

# set objective function
model.setObjective(plan.prod(distr_cost), GRB.MINIMIZE)

Academic license - for non-commercial use only - expires 2021-06-18
Using license file C:\Users\Huyen\gurobi.lic


In [5]:
# Define constraints
# maximal factory supply
factories = f.keys()
factory_con = model.addConstrs((gp.quicksum(plan.select(i, '*')) <= f[i] for i in factories), name="factory_constraint")

# maximal depot throughput
depots = d.keys()
depot_through_con = model.addConstrs((gp.quicksum(plan.select(j, '*')) <= d[j] for j in depots), 
                                     name="depot_through_constraint")
# depot output == depot input
depot_out_con = model.addConstrs((gp.quicksum(plan.select('*', j)) == gp.quicksum(plan.select(j, '*')) for j in depots),
                                                                                 name="depot_out_constraint")
# customer requirement == customer input
customers = c.keys()
customer_con = model.addConstrs((gp.quicksum(plan.select('*', k)) == c[k] for k in customers), name="customer_constraint")

In [6]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0x19130edf
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 15 rows, 28 columns, 73 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6000000e+05   2.624375e+04   0.000000e+00      0s
       6    2.3250000e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds
Optimal objective  2.325000000e+05


In [7]:
# Optimal value of the objective function which is the weighted sum of two objective functions (in €)
obj = model.getObjective()
print("Total distribution costs per month: {0} €".format(obj.getValue()))

Total distribution costs per month: 232500.0 €


In [8]:
# Optimal transportation plan per month (quantities in tons)
trans_plan = pd.DataFrame(columns=["Supplier", "Receiver", "Quantity (in tons)"])
for a in arcs:
    if plan[a].x > 1e-6:
        trans_plan = trans_plan.append({"Supplier": a[0], "Receiver": a[1], "Quantity (in tons)": plan[a].x}, 
                                       ignore_index=True)
trans_plan.index = [''] * len(trans_plan)
trans_plan

,Supplier,Receiver,Quantity (in tons)
,Liverpool,Exeter,40000.0
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,50000.0
,Brighton,London,55000.0
,Birmingham,C2,10000.0
,Birmingham,C4,35000.0
,Birmingham,C5,5000.0
,London,C5,55000.0
,Exeter,C3,40000.0


#### w=0.4
* objective function 1 = 232500 - 55000*w=210500  (€)
* not satisfied customer C5 with the number of tons: 55000

### Check the weight 0.5 for objective function 2

In [11]:
w=0.5
# Dictionary for distribution costs per factory (in €/ton) for the sum of 2 objective functions 
arcs, distr_cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0+w,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'C1'): 2.0+w,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5+w,
    ('Newcastle', 'C6'): 1.0+w,
    ('Birmingham', 'C1'): 1.0+w,
    ('Birmingham', 'C2'): 0.5+w,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5+w,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5+w,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5+w,
    ('Exeter', 'C6'): 1.5    
})

In [12]:
# Define model
model = Model('Transshipment_Assignment')

# Create decision variables
plan = model.addVars(arcs, name="plan")

# set objective function
model.setObjective(plan.prod(distr_cost), GRB.MINIMIZE)

In [13]:
# Define constraints
# maximal factory supply
factories = f.keys()
factory_con = model.addConstrs((gp.quicksum(plan.select(i, '*')) <= f[i] for i in factories), name="factory_constraint")

# maximal depot throughput
depots = d.keys()
depot_through_con = model.addConstrs((gp.quicksum(plan.select(j, '*')) <= d[j] for j in depots), 
                                     name="depot_through_constraint")
# depot output == depot input
depot_out_con = model.addConstrs((gp.quicksum(plan.select('*', j)) == gp.quicksum(plan.select(j, '*')) for j in depots),
                                                                                 name="depot_out_constraint")
# customer requirement == customer input
customers = c.keys()
customer_con = model.addConstrs((gp.quicksum(plan.select('*', k)) == c[k] for k in customers), name="customer_constraint")

In [14]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0x630dd53c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 15 rows, 28 columns, 73 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6300000e+05   2.624375e+04   0.000000e+00      0s
       9    2.4100000e+05   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.03 seconds
Optimal objective  2.410000000e+05


In [15]:
# Optimal value of the objective function which is the weighted sum of two objective functions (in €)
obj = model.getObjective()
print("Total distribution costs per month: {0} €".format(obj.getValue()))

Total distribution costs per month: 241000.0 €


In [16]:
# Optimal transportation plan per month (quantities in tons)
trans_plan = pd.DataFrame(columns=["Supplier", "Receiver", "Quantity (in tons)"])
for a in arcs:
    if plan[a].x > 1e-6:
        trans_plan = trans_plan.append({"Supplier": a[0], "Receiver": a[1], "Quantity (in tons)": plan[a].x}, 
                                       ignore_index=True)
trans_plan.index = [''] * len(trans_plan)
trans_plan

,Supplier,Receiver,Quantity (in tons)
,Liverpool,Newcastle,10000.0
,Liverpool,Exeter,35000.0
,Liverpool,C1,50000.0
,Liverpool,C4,35000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,50000.0
,Brighton,London,10000.0
,Brighton,Exeter,5000.0
,Newcastle,C2,10000.0
,Birmingham,C5,50000.0


#### w=0.5
* objective function 1 = 241000 - 10000*w=236000  (€)
* not satisfied customer C5 with the number of tons: 10000

### Remark:
This is two optimal solutions found by weighted method. Mathias has found another interesting solution by deleting a certain arcs after having some observations from the data. I think we should create a table of the cost and the amount of tons that is shipped by non-prefered as a references for the company.